In [ ]:

import re
from uszipcode import SearchEngine
from uszipcode import Zipcode
from io import StringIO
from csv import writer 

search = SearchEngine(simple_zipcode=True)
result_u = search.by_coordinates(42.2052,-75.5354,radius=10,returns=100)
result_l = search.by_coordinates(39.5478,-78.26556 ,radius=10,returns=100)
                                 
def getCensus_data(census_results):
    x = census_results
    x = str(x)
    check = set([])
    words = ['[SimpleZipcode', "(", ")", "SimpleZipcodezipcode"] #remove the string that the function returns
    for i in words: x = x.replace(i, "")
    t = x.split(",")
    push_row= []
    column_names=['zipcode', 'zipcode_type', 'major_city', 'post_office_city', 'state', 'common_city_list', 'county', 'lat', 'lng', 'timezone', 'radius_in_miles', 'area_code_list', 'population', 'population_density', 'land_area_in_sqmi', 'water_area_in_sqmi', 'housing_units', 'occupied_housing_units', 'median_home_value', 'median_household_income', 'bounds_west', 'bounds_east', 'bounds_north', 'bounds_south']
    column_set= ['zipcode', 'zipcode_type', 'major_city', 'post_office_city', 'state', 'common_city_list', 'county', 'lat', 'lng', 'timezone', 'radius_in_miles', 'area_code_list', 'population', 'population_density', 'land_area_in_sqmi', 'water_area_in_sqmi', 'housing_units', 'occupied_housing_units', 'median_home_value', 'median_household_income', 'bounds_west', 'bounds_east', 'bounds_north', 'bounds_south']
    #turn this into a string io so that we can add to dataframe
    ## attempted hash table, but data would split weird
    output = StringIO()
    csv_writer = writer(output)
    for i in t:

        try:
            a,b = i.split("=")
            a, b = a.strip(), b.strip()
           # print(a,b)
        except ValueError:
            b = i
            b = b.strip()
            if(len(b)==2):
                a = "state"
            else: 
                a = "NOT this"

        
        if((a in column_set) and (a!="bounds_south")):
            push_row.append(b)   
            
        elif(a == "bounds_south"):## found the end of the row
            

            push_row.append(b) 
            csv_writer.writerow(push_row)
            push_row = []
        
            ##there is a list in the code. 
    output.seek(0) 

    # we need to get back to the start of the BytesIO
    df = pd.read_csv(output, error_bad_lines=False, header= None) 
    df.columns = column_names
    return df
